In [16]:
from torch2trt import TRTModule

ModuleNotFoundError: No module named 'torch2trt'

In [17]:
!pip install torch2trt

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement torch2trt (from versions: none)
ERROR: No matching distribution found for torch2trt


In [2]:
import torchvision
import torch

# road following
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)

# collision avoidance
model_ca = torchvision.models.alexnet(pretrained=False)
model_ca.classifier[6] = torch.nn.Linear(model_ca.classifier[6].in_features, 2)

In [3]:
model_rf.load_state_dict(torch.load('best_steering_model_xy.pth')) # well trained road following model
model_ca.load_state_dict(torch.load('best_model.pth')) # well trained collision avoidance model

<All keys matched successfully>

In [4]:
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

model_ca = model_ca.to(device)

In [5]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess_rf(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

# collision avoidance
mean_ca = 255.0 * np.array([0.485, 0.456, 0.406])
stdev_ca = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean_ca, stdev_ca)

def preprocess_ca(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [6]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)



In [7]:
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
from jetbot import Robot

robot = Robot()

In [9]:
#Road Following sliders
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.04, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

#Collision Avoidance sliders
blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stopduration_slider= ipywidgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop') 
blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

display(image_widget)

display(ipywidgets.HBox([blocked_slider, blocked_threshold, stopduration_slider]))

FloatSlider(value=0.0, description='speed control', max=1.0, step=0.01)

FloatSlider(value=0.04, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [13]:
import math

angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = speed_control_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
    x = change['new'] 
    x = preprocess_ca(x)
    y = model_ca(x)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(y, dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            image = change['new']
            xy = model_rf(preprocess_rf(image)).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
            
    
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steer_val = pid + steer_bias 
    angle_last = angle
    robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0) 

execute({'new': camera.value})

In [14]:
camera.observe(execute, names='value')

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg/jetbot/camera.py", line 45, in _capture_frames
    self.value = image
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 125

In [15]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()